This file will follow tutorial 
https://www.depends-on-the-definition.com/named-entity-recognition-conditional-random-fields-python/

In [ ]:
import pandas as pd
import numpy as np

In [ ]:
def read_all_line( directory , file ):
    file = open( directory + "/" + file , "r" , encoding="utf-8" )
    contents = file.read()
    file.close()
    return contents

In [ ]:
dictionary_directory = "dictionary_directory"
dictionary_files = {
    "front_person" : ("clue front.txt" , "clue word person.txt" , "clue_royal.txt")
    ,"front_country" : ("คำนำหน้าชื่อประเทศ.txt")
    ,"front_org" : ("คำนำหน้าองค์กรจาก dict.txt")
    ,"location_name" : ("ชื่อกิ่งอำเภอ.txt" , "ชื่อคลอง.txt" , "ชื่อจังหวัด.txt" , "ชื่อตำบล.txt" , "ชื่อมลรัฐ.txt" , "ชื่อสถานที่.txt")
}

In [ ]:
def prepare_dictionary( directory , file_dict ):
    dictionary = {}
    for key in file_dict.keys():
        dictionary[ key ] = []
        list_file = file_dict[ key ]
        if type( list_file ) == type( "test_string" ):
            dictionary[ key ] = read_all_line( directory , list_file ).split('\n')
        else:
            for file in list_file:
                dictionary[ key ] += read_all_line( directory , file ).split('\n')
    return dictionary

# If found in dict will return key other return NO
def search_in_dictionary( word , dictionary ):
    answer = "NO"
    for key in dictionary.keys():
        if word in dictionary[key]:
            answer = key
            break
    return answer

In [ ]:
dictionary = prepare_dictionary( dictionary_directory , dictionary_files )
search_in_dictionary( "และ" , dictionary)

In [ ]:
def idividual_read_file( raw_data , split_sentence , count ,remove = [] ):
    pre_data_frame = { "Sentence #" : [] , "Word" : [] , "Tag" : [] , "Dictionary" : [] }
    for word in raw_data:
        if word in remove :
            None
        elif word in split_sentence :
            count += 1
        else: 
            check = word.find('(')
            if check > 0:
                temp_word = word[ 0 : word.find('(')]
                pre_data_frame["Sentence #"].append( "Sentence: " + str(count) )
                pre_data_frame["Word"].append( temp_word )
                pre_data_frame["Tag"].append( word[ word.find('(') + 1 : word.find(')')] )
                pre_data_frame["Dictionary"].append( search_in_dictionary( temp_word , dictionary ) )
            else:
                pre_data_frame["Sentence #"].append( "Sentence: " + str(count) )
                pre_data_frame["Word"].append( word )
                pre_data_frame["Tag"].append( "O" )
                pre_data_frame["Dictionary"].append( search_in_dictionary( word , dictionary ) )
    return pre_data_frame , count

def read_file( directory , list_file , count = 1 , individual = False ):
    data_frame = pd.DataFrame( { "Sentence #" : [] , "Word" : [] , "Tag" : [] , "Dictionary" : []} )
    if( individual ):
        word = read_all_line( directory , list_file ).split('|')
        pre_data_frame , count = idividual_read_file( word , # raw_data
                                              ["\n"] , # word show split sentence
                                              count , # order of sentence
                                              [ " " , '' , '\0'] ) # word to delete or prevent
        data_frame  = data_frame.append( pd.DataFrame( pre_data_frame ), ignore_index=True )
    else:
        for file in list_file :
            word = read_all_line( directory , file ).split('|')
            pre_data_frame , count = idividual_read_file( word ,
                                              ["\n"] ,
                                              count ,
                                              [ " " , '' , '\0'] )
            data_frame  = data_frame.append( pd.DataFrame( pre_data_frame ), ignore_index=True )
    return data_frame
def num_of_sentence( sentence ):
    words = sentence.split( " ")
    return int( words[1] )

In [ ]:
# Variable for collect name file and access file
train_directory = "corpus_directory"
train_files = ("Politic9.txt" , "Allcolumn.txt" , "98JUL5_1.txt" ,
                "98JUL5_2.txt" , "bkknews1.txt" , "Result1.txt" ,
                "Result2(corpus-1).txt" , "Result3(corpus-2).txt" )
test_directory = "corpus_directory"
test_files = ("result4.txt" , "Result_c_2_1.txt", "Result_c_2_2.txt" )

In [ ]:
data = read_file( train_directory , train_files , 1 )
data.fillna( method="ffill")
data.tail( 10 )

In [ ]:
def merge_all_tag( data ):
    for run in range(  0 , data.shape[0] ):
        if data.Tag[run] in ( '' ):
            data.Tag[run] = 'O'
        elif data.Tag[run] in ('loc_cont','loc_end','loc_start'):
            data.Tag[run] = 'loc'
        elif data.Tag[run] in ('org_cont','org_end','org_start'):
            data.Tag[run] = 'org'
        elif data.Tag[run] in ('per_cont','per_end','per_start'):
            data.Tag[run] = 'per'
        else:
            None

# number is current index
# target is -1 -2 +1 +2 what do you want to file
def helper_get_tag( data , number , target , limit ):
    if( number + target == limit ):
        return 'O'
    elif( data["Sentence #"][ number+target ] != data["Sentence #"][ number ] ):
        return 'O'
    else:
        return data.Tag[number + target ]
            
def split_all_tag( data ):
    limit_run = data.shape[0]
    for run in range( 0 , limit_run ):
        if data.Tag[ run ] == "loc" :
            if( helper_get_tag( data , run , -1, limit_run  ) in ( "loc_start" , "loc_cont" ) ):
                if( helper_get_tag( data , run , +1 , limit_run ) == "loc" ):
                    data.Tag[ run ] = "loc_cont"
                else:
                    data.Tag[ run ] = "loc_end"
            elif( helper_get_tag( data , run , +1 , limit_run ) == "loc" ):
                data.Tag[ run ] = "loc_start"
        elif data.Tag[ run ] == "org" :
            if( helper_get_tag( data , run , -1 , limit_run ) in ( "org_start" , "org_cont" ) ):
                if( helper_get_tag( data , run , +1 , limit_run ) == "org" ):
                    data.Tag[ run ] = "org_cont"
                else:
                    data.Tag[ run ] = "org_end"
            elif( helper_get_tag( data , run , +1 , limit_run ) == "org" ):
                data.Tag[ run ] = "org_start"
        elif data.Tag[ run ] == "per" :
            if( helper_get_tag( data , run , -1 , limit_run ) in ( "per_start" , "per_cont" ) ):
                if( helper_get_tag( data , run , +1, limit_run ) == "per" ):
                    data.Tag[ run ] = "per_cont"
                else:
                    data.Tag[ run ] = "per_end"
            elif( helper_get_tag( data , run , +1 , limit_run ) == "per" ):
                data.Tag[ run ] = "per_start"
            else:
                None
        else:
            None

In [ ]:
merge_all_tag( data )
words = list( set(data["Word"].values ) )

In [ ]:
n_words = len( words )
n_sentence = num_of_sentence( data["Sentence #"][ data.shape[0]-1 ] )
print( "You have number of words is {} from {} sentences".format( n_words , n_sentence ))

In [ ]:
class SentenceGetter(object):
    
    def __init__(self, data):
        self.n_sent = 0
        self.data = data
        self.empty = False
        agg_func = lambda s: [(w, t, d) for w, t, d in zip( s["Word"].values.tolist(),
                                                            s["Tag"].values.tolist(),
                                                            s["Dictionary"].values.tolist() 
                                                          )]
        self.grouped = self.grouped_sentence()
        self.sentences = [s for s in self.grouped]
    
    def grouped_sentence( self ):
        save_sentence = 0
        major_index = 0
        temp_grouped = []
        for run in range( 0 , self.data.shape[0] ):
            temp_sentence = num_of_sentence( self.data["Sentence #"][ run ] )
            if save_sentence != temp_sentence :
                major_index = temp_sentence - 1
                save_sentence = temp_sentence
                temp_grouped.append( [] )
            temp_grouped[ major_index ].append( (self.data["Word"][run], 
                                                 self.data["Tag"][run],
                                                 self.data["Dictionary"][run] ) )
        return temp_grouped
    
    def get_next(self):
        try:
            s = self.grouped[ self.n_sent ]
            self.n_sent += 1
            return s  
        except:
            self.empty = True
            return None, None

In [ ]:
getter = SentenceGetter( data )

In [ ]:
sent= getter.get_next()

In [ ]:
print(sent)

ข้างบนนั้นจะดึงออกมาทีละประโยค โดยที่จะดึงคำออกมาพร้อมกับ tag นั้นเอง
ดังนั้นต่อมาเราจะดึงออกทั้งหมด ทุกประโยคเลย

In [ ]:
sentences = getter.sentences

In [ ]:
def word2features(sent, i):
    word = sent[i][0]
    dict_type = sent[i][2]

    features = {
        'bias': 1.0,
        'word.lower()': word.lower(),
        'word[-3:]': word[-3:],
        'word[-2:]': word[-2:],
        'word.isupper()': word.isupper(),
        'word.istitle()': word.istitle(),
        'word.isdigit()': word.isdigit(),
        'dictionary' : dict_type
    }
    if i > 0:
        word1 = sent[i-1][0]
        dict_type1 = sent[i-1][2]
        features.update({
            '-1:word.lower()': word1.lower(),
            '-1:word.istitle()': word1.istitle(),
            '-1:word.isupper()': word1.isupper(),
            '-1:dictionay' : dict_type1
        })
    else:
        features['BOS'] = True

    if i < len(sent)-1:
        word1 = sent[i+1][0]
        dict_type1 = sent[i+1][2]
        features.update({
            '+1:word.lower()': word1.lower(),
            '+1:word.istitle()': word1.istitle(),
            '+1:word.isupper()': word1.isupper(),
            '-1:dictionay' : dict_type1
        })
    else:
        features['EOS'] = True

    return features


def sent2features(sent):
    return [word2features(sent, i) for i in range(len(sent))]

def sent2labels(sent):
    return [label for token, label, dict_type in sent]

def sent2tokens(sent):
    return [token for token, label, dict_type in sent]

def sent2dict_type( sent ):
    return [dict_type for token, label, dict_type in sent]

ข้างบนเป็นการจัดรูปแบบ feature ของคำเฉยๆ ถ้าอยากรู้จะต้องอ่านเนื้อหาเกี่ยวกับ CRF เพิ่มเติมว่ามีแนวคิดอย่างไร คาดว่าจุดนี้อาจทำให้สามารถคำนวณโดยเอา Dict มาใช้ได้
https://medium.com/ml2vec/overview-of-conditional-random-fields-68a2a20fa541

In [ ]:
from sklearn_crfsuite import CRF

crf = CRF(algorithm='lbfgs',
          c1=0.1,
          c2=0.1,
          max_iterations=100,
          all_possible_transitions=False)

In [ ]:
# ref for first module https://scikit-learn.org/stable/modules/cross_validation.html
from sklearn.model_selection import cross_val_predict
from sklearn.metrics import classification_report
from sklearn_crfsuite.metrics import flat_classification_report

In [ ]:
X = [sent2features(s) for s in sentences]
y = [sent2labels(s) for s in sentences]

In [ ]:
pred = cross_val_predict(estimator=crf, X=X, y=y, cv=5)

In [ ]:
report = flat_classification_report(y_pred=pred, y_true=y)
print(report)

In [ ]:
crf.fit(X, y)

In [ ]:
#import eli5

In [ ]:
#eli5.show_weights(crf, top=30)

In [ ]:
# reference https://machinelearningmastery.com/save-load-machine-learning-models-python-scikit-learn/
import pickle
model_directory = "model_directory"
model_name = "CRF_dictionary.sav"
model_file = model_directory + "/" + model_name

In [ ]:
pickle.dump( crf , open( model_file , 'wb') )

In [ ]:
load_model = pickle.load( open( model_file , 'rb') )

In [ ]:
load_model

In [ ]:
data_test = read_file( test_directory , test_files , 1 )
data_test.fillna( method="ffill")
data_test.tail( 10 )

In [ ]:
merge_all_tag( data_test )

In [ ]:
getter_test = SentenceGetter( data_test )

In [ ]:
getter_test.get_next()

In [ ]:
sentences_test = getter_test.sentences
sentences_test[1]

In [ ]:
test_set = [sent2features(s) for s in sentences_test]
test_label = [sent2labels(s) for s in sentences_test]

In [ ]:
test_predict = load_model.predict( test_set )

In [ ]:
report = flat_classification_report(y_pred=test_predict, y_true=test_label)
print(report)

In [ ]:
data_test.head( 20 )

In [ ]:
def convert_list_to_data_frame( data_frame , data_list ):
    save_sentence = 0
    major_index_list = 0
    minor_index_list = 0
    for run in range( 0 , data_frame.shape[0] ):
        temp_sentence = num_of_sentence( data_frame["Sentence #"][ run ] )
        if save_sentence != temp_sentence :
            major_index_list = temp_sentence - 1
            save_sentence = temp_sentence
            minor_index_list = 0
        print( "access [{}][{}]".format( major_index_list , minor_index_list ))
        data_frame["Tag"][run] = data_list[ major_index_list][ minor_index_list ]
        minor_index_list += 1

In [ ]:
convert_list_to_data_frame( data_test , test_predict )

In [ ]:
data_test

In [ ]:
split_all_tag( data_test )

In [ ]:
data_test